# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-6. rnn_fc_stock_prediction

<hr>
``` python 

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility


def MinMaxScaler(data):
    num_row = np.shape(data)[0]
    num_col = np.shape(data)[1]
    array = np.zeros((num_row, num_col))
    for i in range(num_col):
        input = data[:, i]
        array[:, i] = (input - np.min(input)) / (np.max(input) - np.min(input))
    return array

timesteps = seq_length = 7
data_dim = 5
output_dim = 3

# Open,High,Low,Close,Volume
xy = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1]  # reverse order (chronically ordered)
xy = MinMaxScaler(xy)

x = xy
y = xy[:, [-1]]  # Close as label

dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price as target
    print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

# split to train and testing
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=output_dim, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
last_output = outputs[:, -1]  # We use the last cell's output

# Fully connected layer (rnn_hidden_size -> num_classes)
fc_w = tf.get_variable("fc_w", [output_dim, 1])
fc_b = tf.get_variable("fc_b", [1])
Y_pred = tf.matmul(last_output, fc_w) + fc_b

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    _, l = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    print(i, l)

testPredict = sess.run(Y_pred, feed_dict={X: testX})
print("RMSE", sess.run(rmse, feed_dict={
      targets: testY, predictions: testPredict}))
plt.plot(testY)
plt.plot(testPredict)
plt.show()


```

<hr>

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility

C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def MinMaxScaler(data):
    num_row = np.shape(data)[0]
    num_col = np.shape(data)[1]
    array = np.zeros((num_row, num_col))
    for i in range(num_col):
        input = data[:, i]
        array[:, i] = (input - np.min(input)) / (np.max(input) - np.min(input))
        
    return array


In [5]:
timesteps = seq_length = 7
data_dim = 5
output_dim = 3

# Open,High,Low,Close,Volume
xy = np.loadtxt('./data/data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1]  # reverse order (chronically ordered)
xy = MinMaxScaler(xy)

x = xy
y = xy[:, [-1]]  # Close as label

In [6]:
x.shape, y.shape

((732, 5), (732, 1))

In [7]:
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price as target
    # print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)


In [8]:
dataX[:3], dataY[:3]

([array([[2.13751054e-01, 2.08179810e-01, 1.91791832e-01, 4.66075110e-04,
          1.92092403e-01],
         [1.93935034e-01, 2.03641926e-01, 2.08664571e-01, 2.98467330e-03,
          1.96551555e-01],
         [2.10516454e-01, 2.05289413e-01, 2.03558748e-01, 2.59926504e-04,
          1.87749731e-01],
         [1.86678765e-01, 2.09480567e-01, 2.08781843e-01, 0.00000000e+00,
          2.17448151e-01],
         [3.06973882e-01, 3.14634137e-01, 2.18993665e-01, 1.24764722e-02,
          2.16981885e-01],
         [2.19142111e-01, 2.63907214e-01, 2.24686396e-01, 4.56323384e-01,
          2.24967474e-01],
         [2.33129931e-01, 2.36419163e-01, 1.62682724e-01, 5.70171193e-01,
          1.47442742e-01]]),
  array([[1.93935034e-01, 2.03641926e-01, 2.08664571e-01, 2.98467330e-03,
          1.96551555e-01],
         [2.10516454e-01, 2.05289413e-01, 2.03558748e-01, 2.59926504e-04,
          1.87749731e-01],
         [1.86678765e-01, 2.09480567e-01, 2.08781843e-01, 0.00000000e+00,
          2.174

In [9]:
# split to train and testing
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=output_dim, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
last_output = outputs[:, -1]  # We use the last cell's output

# Fully connected layer (rnn_hidden_size -> num_classes)
fc_w = tf.get_variable("fc_w", [output_dim, 1])
fc_b = tf.get_variable("fc_b", [1])
Y_pred = tf.matmul(last_output, fc_w) + fc_b

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))


ValueError: Dimensions must be equal, but are 6 and 8 for 'rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul_1' (op: 'MatMul') with input shapes: [?,6], [8,12].

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in tqdm_notebook(range(1000+1)):
    _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
       
    if i<10 or i%10==0:
        print("Step : {} \t Loss : {}".format(i, step_loss))    

In [ ]:
testPredict = sess.run(Y_pred, feed_dict={X: testX})
print("RMSE : ", sess.run(rmse, feed_dict={
      targets: testY, predictions: testPredict}))


In [ ]:
# %matplotlib notebook
# plt.rcParams["figure.figsize"] = [10,6]

%matplotlib inline
plt.rcParams["figure.figsize"] = [16,8]

plt.plot(testY, label='Time')
plt.plot(testPredict, label='Price')
plt.legend()
plt.grid()
plt.title('Stock Price Prediction')
plt.xlabel("Time Period")
plt.ylabel("Stock Price")
plt.show()


<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>